## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



They use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [1]:
using StatsBase
using Distributions
using Plots
using IJulia

In [2]:
using TensorFlow
using MLDataUtils

In [3]:
using DensityEstimationML
function only(itr)
    state = start(itr)
    val,state = next(itr, state)
    @assert(done(itr,state))
    return val
end

only (generic function with 1 method)

In [4]:
immutable NeuralDensityEstimator
    sess::Session
    
    #Network nodes
    optimizer::Tensor
    conditioner::Tensor
    t::Tensor
    pdf::Tensor
end

In [5]:
function Distributions.pdf(est::NeuralDensityEstimator, t::Real)
    ts = reshape([t], (1,1))
    pdf(est, ts) |> first
end

function Distributions.pdf(est::NeuralDensityEstimator, ts::AbstractVector)
    gr = est.sess.graph
    run(est.sess, est.pdf, Dict(est.t=>ts')) |> vec
end

function Distributions.loglikelihood(est::NeuralDensityEstimator, ts::AbstractVector)
    gr = est.sess.graph
    run(est.sess, gr["loglikelihood"], Dict(est.t=>ts')) |> vec
end


In [43]:
"""
Function returning a function that will display a running plot.
WARNING: Introducting or removing any variables is not supported.
And will silently error.
"""
function running_plot()
    epochs = Int[]
    record = Dict()
    function inner(epoch, vars::Associative)
        for (var, values) in vars
            value = only(values) #Incase it was an array
            past = get!(record, var) do
                typeof(value)[]
            end
            push!(past, value)
        end
        push!(epochs, epoch)
        
        IJulia.clear_output(true)
        plot(epochs, hcat(values(record)...); label=hcat(keys(vars)...), layout=length(vars)) |> IJulia.display       
    end
end


running_plot

In [44]:
ignore(epoch, vars_dict) = nothing

function StatsBase.fit!(estimator::NeuralDensityEstimator, observations;
    callback=ignore, callback_vars=["ysmin", "ysmax", "loglikelihood", "working_loss"],
    epochs = 20)
    
    gr = estimator.sess.graph
    callback_nodes = [gr[var] for var in callback_vars]
        
        
    
    for ii in 1:epochs
        outs = run(estimator.sess, 
            [callback_nodes..., estimator.optimizer],
            Dict(estimator.t=>observations'))
        
        if ii % 100 == 1
            callback(ii, Dict(zip(callback_vars, outs[1:end-1])))
        end
        
    end
    estimator
end

In [45]:
function NeuralDensityEstimator(prob_layer_sizes, support)
    sess = Session(Graph())
    @tf begin
        t = placeholder(Float32, shape=[1, -1])
        smin = constant(reshape([minimum(support)],(1,1)))
        smax = constant(reshape([maximum(support)],(1,1)))
        
        layer_sizes= [1; prob_layer_sizes; 1]
        
        network_fun_stack = Function[Base.identity]       
       
        for ii in 2:length(layer_sizes)
            below_size = layer_sizes[ii-1]
            above_size = layer_sizes[ii]
                       
            Wii = get_variable("W_$ii", [above_size, below_size], Float32)
            Wii2  = Ops.mul(Wii, Wii; name = "W_$(ii)_squared")
            
            #bii = get_variable("b_$ii", [above_size, 1], Float32)
            #act_fun = z -> nn.sigmoid(Wii2*z .+ bii)
            
            act_fun = if ii!=length(layer_sizes)
                bii = get_variable("b_$ii", [above_size, 1], Float32)
                z -> nn.sigmoid(Wii2*z .+ bii)
            else
                z-> exp(Wii2*z)
            end
            push!(network_fun_stack, z->act_fun(network_fun_stack[ii-1](z)))
        end
        
        network = network_fun_stack[end]

        
        ysmin = TensorFlow.identity(network(smin))
        ysmax = TensorFlow.identity(network(smax))
        yt = network(t)
        
        denominator = (ysmax-ysmin) #area
        numerator = gradients(yt,t)
        pdf = numerator/denominator
        
        
        n_points = TensorFlow.shape(t)[2]
        loglikelihood = reduce_sum(log(numerator)) - n_points.*log(denominator)
        
        area_loss = (1f0.-denominator)^2
        working_loss = -1*loglikelihood + 0.1*area_loss
        
        optimizer = train.minimize(train.AdamOptimizer(), working_loss)
        
        
        # Conditioning
        # Make sure that ysmin~=1, and ysmax~=2
        condition_loss = (1f0 - ysmin)^2 + (2f0 - ysmax)^2
        condition_optimiser = train.minimize(train.AdamOptimizer(;name="adam_cond"), condition_loss)
    end
    
    run(sess, global_variables_initializer())
    
    NeuralDensityEstimator(sess, optimizer, condition_optimiser, t, pdf)
end

NeuralDensityEstimator

In [46]:
"""
    condition(est::NeuralDensityEstimator tol = 1e-15, max_epochs=2_000)
    
"Conditions" the neural density estimate so the support extrema are mapped to 1. and 2.
This improves training by adjusting the area the network has the learn over

"""
function condition!(est::NeuralDensityEstimator, tol = 1e-15, max_epochs=2_000)
    gr = est.sess.graph
    for ii in 1:2_000
        _, ysmin, ysmax, condition_loss = run(est.sess, [est.conditioner, gr["ysmin"],gr["ysmax"], gr["condition_loss"]])
        ii % 50 == 1 && @show (ii, ysmin, ysmax, condition_loss)
        if condition_loss[1] < 1e-15
            break
        end
    end
end


condition!

In [47]:
function demonstration_plot(est, dataset, data=rand(dataset), args...; kwargs...)
    X = minimum(approximate_support(dataset)) : 0.01 : maximum(approximate_support(dataset))
    println("True loglikelihood      = $(loglikelihood(dataset, data))")
    println("Estimated loglikelihood = $(loglikelihood(est, data))")
    estimated_loglikelihood = loglikelihood(dataset, data)
    plot([X], [pdf(est,X), data],
        #xlims= approximate_support(dataset),
        xlims= (first(X), last(X)),
        seriestype = [:path :histogram],
        layout=(2,1),
        legend=false,
        nbins=[1  length(data)÷10],
        args...; kwargs...
    )
end

demonstration_plot (generic function with 2 methods)

In [48]:
function demo(dataset, layers, epochs=20_000)
    data = original_sample(dataset)
    @show loglikelihood(dataset, data)
    est = NeuralDensityEstimator(layers, approximate_support(dataset))
    condition!(est)
    println("Conditioning Done")
    fit!(est, data; epochs=epochs, callback=running_plot())
    println("Fitting Done")
    demonstration_plot(est, dataset, data)
end

demo (generic function with 2 methods)

In [49]:
demo(GenerateDatasets.Likas1(), [64,64], 20_000)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 5 
 
 
 10 
 
 
 15 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.6177,175.813 56.7642,164.512 57.9107,160.624 59.0572,155.49 60.2038,152.255 61.3503,151.651 62.4968,151.7 63.6433,151.11 64.7898,150.922 65.9364,148.614 
 67.0829,129.541 68.2294,93.0659 69.3759,55.99 70.5225,32.035 71.669,17.2117 72.8155,11.811 73.962,16.4766 75.1085,26.6228 76.2551,34.2748 77.4016,40.8624 
 78.5481,46.9734 79.6946,53.3301 80.8412,59.6891 81.9877,65.8407 83.1342,71.6543 84.2807,76.9866 85.4273,81.7882 86.5738,85.927 87.7203,89.6207 88.8668,93.1072 
 90.0133,96.3146 91.1599,99.2261 92.3064,101.859 93.4529,104.229 94.5994,106.351 95.746,108.241 96.8925,109.919 98.039,111.403 99.1855,112.716 100.332,113.88 
 101.479,114.916 102.625,115.84 103.772,116.668 104.918,117.423 106.065,118.137 107.211,118.828 108.358,119.506 109.504,120.169 110.651,120.816 111.797,121.446 
 112.944,122.061 114.09,122.663 115.237,123.256 116.383,123.851 117.53,124.451 118.676,125.058 119.823,125.673 120.969,126.296 122.116,126.928 123.263,127.564 
 124.409,128.203 125.556,128.848 126.702,129.516 127.849,130.225 128.995,130.991 130.142,131.812 131.288,132.679 132.435,133.576 133.581,134.504 134.728,135.464 
 135.874,136.455 137.021,137.486 138.167,138.546 139.314,139.635 140.46,140.751 141.607,141.915 142.753,143.108 143.9,144.314 145.046,145.518 146.193,146.695 
 147.339,147.851 148.486,148.979 149.633,150.082 150.779,151.148 151.926,152.172 153.072,153.13 154.219,154.004 155.365,154.79 156.512,155.483 157.658,156.1 
 158.805,156.646 159.951,157.139 161.098,157.598 162.244,158.047 163.391,158.513 164.537,158.733 165.684,155.654 166.83,155.727 167.977,156.216 169.123,156.714 
 170.27,157.223 171.416,157.745 172.563,158.268 173.71,158.769 174.856,159.254 176.003,159.719 177.149,160.164 178.296,160.595 179.442,161.008 180.589,161.385 
 181.735,161.753 182.882,162.124 184.028,162.487 185.175,162.845 186.321,163.189 187.468,163.527 188.614,163.856 189.761,164.176 190.907,164.487 192.054,164.79 
 193.2,165.085 194.347,165.373 195.493,165.653 196.64,165.922 197.786,166.184 198.933,166.437 200.08,166.681 201.226,166.918 202.373,167.148 203.519,167.369 
 204.666,167.584 205.812,167.791 206.959,167.992 208.105,168.186 209.252,168.372 210.398,168.552 211.545,168.725 212.691,168.89 213.838,169.049 214.984,169.201 
 216.131,169.342 217.277,169.48 218.424,169.607 219.57,169.727 220.717,169.838 221.863,169.934 223.01,170.01 224.157,170.101 225.303,170.192 226.45,170.272 
 227.596,170.331 228.743,170.376 229.889,170.41 231.036,170.437 232.182,170.462 233.329,170.488 234.475,170.525 235.622,170.58 236.768,170.659 237.915,170.76 
 239.061,170.875 240.208,170.997 241.354,171.117 242.501,171.233 243.647,171.344 244.794,171.451 245.94,171.551 247.087,171.647 248.233,171.743 249.38,171.839 
 250.527,171.931 251.673,172.02 252.82,172.104 253.966,172.183 255.113,172.268 256.259,172.371 257.406,172.362 258.552,172.238 259.699,172.3 260.845,172.41 
 261.992,172.529 263.138,172.651 264.285,172.769 265.431,172.882 266.578,172.984 267.724,173.083 268.871,173.177 270.017,173.264 271.164,173.346 272.31,173.423 
 273.457,173.497 274.604,173.568 275.75,173.634 276.897,173.7 278.043,173.765 279.19,173.826 280.336,173.887 281.483,173.946 282.629,174.005 283.776,174.06 
 
 "/>
 
 
 
 
 ysmax 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 15000 
 
 
 20000 
 
<polyline clip-path="url(#clip03)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 352.157,11.811 353.303,86.9863 354.45,103.545 355.596,108.417 356.743,109.633 357.89,110.325 359.036,110.946 360.183,111.672 361.329,113.004 362.476,115.415 
 363.622,118.9 364.769,122.925 365.915,128.961 367.062,133.132 368.208,134.642 369.355,135.615 370.501,138.318 371.648,143.196 372.794,147.141 373.

Fitting Done
True loglikelihood      = -10534.497259203557
Estimated loglikelihood = [-10601.6]


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.8117,175.813 39.0374,175.813 39.263,175.813 39.4886,175.813 39.7142,175.813 39.9399,175.813 40.1655,175.813 40.3911,175.813 40.6167,175.813 40.8424,175.813 
 41.068,175.813 41.2936,175.813 41.5193,175.813 41.7449,175.813 41.9705,175.813 42.1961,175.813 42.4218,175.813 42.6474,175.813 42.873,175.813 43.0986,175.813 
 43.3243,175.813 43.5499,175.813 43.7755,175.813 44.0011,175.813 44.2268,175.813 44.4524,175.813 44.678,175.813 44.9036,175.813 45.1293,175.813 45.3549,175.813 
 45.5805,175.813 45.8062,175.813 46.0318,175.813 46.2574,175.813 46.483,175.813 46.7087,175.813 46.9343,175.813 47.1599,175.813 47.3855,175.813 47.6112,175.813 
 47.8368,175.813 48.0624,175.813 48.288,175.813 48.5137,175.813 48.7393,175.813 48.9649,175.813 49.1905,175.813 49.4162,175.813 49.6418,175.813 49.8674,175.813 
 50.0931,175.813 50.3187,175.813 50.5443,175.813 50.7699,175.813 50.9956,175.813 51.2212,175.813 51.4468,175.813 51.6724,175.813 51.8981,175.813 52.1237,175.813 
 52.3493,175.813 52.5749,175.813 52.8006,175.813 53.0262,175.813 53.2518,175.813 53.4774,175.813 53.7031,175.813 53.9287,175.813 54.1543,175.813 54.38,175.813 
 54.6056,175.813 54.8312,175.813 55.0568,175.813 55.2825,175.813 55.5081,175.813 55.7337,175.813 55.9593,175.813 56.185,175.813 56.4106,175.813 56.6362,175.813 
 56.8618,175.813 57.0875,175.813 57.3131,175.813 57.5387,175.813 57.7643,175.813 57.99,175.813 58.2156,175.813 58.4412,175.813 58.6669,175.813 58.8925,175.813 
 59.1181,175.813 59.3437,175.813 59.5694,175.813 59.795,175.813 60.0206,175.813 60.2462,175.813 60.4719,175.813 60.6975,175.813 60.9231,175.813 61.1487,175.813 
 61.3744,175.813 61.6,175.813 61.8256,175.813 62.0512,175.813 62.2769,175.813 62.5025,175.813 62.7281,175.813 62.9538,175.813 63.1794,175.813 63.405,175.813 
 63.6306,175.813 63.8563,175.813 64.0819,175.813 64.3075,175.813 64.5331,175.813 64.7588,175.813 64.9844,175.813 65.21,175.813 65.4356,175.813 65.6613,175.813 
 65.8869,175.813 66.1125,175.813 66.3381,175.813 66.5638,175.813 66.7894,175.813 67.015,175.813 67.2407,175.813 67.4663,175.813 67.6919,175.813 67.9175,175.813 
 68.1432,175.813 68.3688,175.813 68.5944,175.813 68.82,175.813 69.0457,175.813 69.2713,175.813 69.4969,175.813 69.7225,175.813 69.9482,175.813 70.1738,175.813 
 70.3994,175.813 70.625,175.813 70.8507,175.813 71.0763,175.813 71.3019,175.813 71.5276,175.813 71.7532,175.813 71.9788,175.813 72.2044,175.813 72.4301,175.813 
 72.6557,175.813 72.8813,175.813 73.1069,175.813 73.3326,175.812 73.5582,175.812 73.7838,175.812 74.0094,175.812 74.2351,175.812 74.4607,175.812 74.6863,175.812 
 74.912,175.812 75.1376,175.812 75.3632,175.812 75.5888,175.812 75.8145,175.812 76.0401,175.812 76.2657,175.812 76.4913,175.812 76.717,175.812 76.9426,175.812 
 77.1682,175.812 77.3938,175.812 77.6195,175.812 77.8451,175.812 78.0707,175.812 78.2963,175.812 78.522,175.812 78.7476,175.812 78.9732,175.812 79.1989,175.812 
 79.4245,175.812 79.6501,175.811 79.8757,175.811 80.1014,175.811 80.327,175.811 80.5526,175.811 80.7782,175.811 81.0039,175.811 81.2295,175.811 81.4551,175.811 
 81.6807,175.811 81.9064,175.811 82.132,175.811 82.3576,175.811 82.5832,175.811 82.8089,175.81 83.0345,175.81 83.2601,175.81 83.4858,175.81 83.7114,175.81 
 83.937,175.81 84.1626,175.81 84.3883,175.81 84.6139,175.81 84.8395,175.809 85.0651,175.809 85.2908,175.809 85.5164,175.809 85.742,175.809 85.9676,175.809 
 86.1933,175.809 86.4189,175.808 86.6445,175.808 86.8701,175.808 87.0958,175.808 87.3214,175.808 87.547,175.807 87.7727,175.807 87.9983,175.807 88.2239,175.807 
 88.4495,175.807 88.6752,175.806 88.9008,175.806 89.1264,175.806 89.352,175.806 89.5777,175.805 89.8033,175.805 90.0289,175.805 90.2545,175.804 90.4802,175.804 
 90.7058,175.804 90.9314,175.803 91.157

In [50]:
demo(GenerateDatasets.Likas2(), [64,64], 20_000)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.6177,175.813 56.7642,175.774 57.9107,175.665 59.0572,175.508 60.2038,175.44 61.3503,175.446 62.4968,175.458 63.6433,175.469 64.7898,175.48 65.9364,175.487 
 67.0829,175.489 68.2294,175.488 69.3759,175.491 70.5225,175.496 71.669,175.505 72.8155,175.514 73.962,175.524 75.1085,175.533 76.2551,175.539 77.4016,175.545 
 78.5481,175.551 79.6946,175.556 80.8412,175.562 81.9877,175.568 83.1342,175.574 84.2807,175.58 85.4273,175.587 86.5738,175.593 87.7203,175.598 88.8668,175.604 
 90.0133,175.609 91.1599,175.614 92.3064,175.619 93.4529,175.624 94.5994,175.63 95.746,175.635 96.8925,175.64 98.039,175.646 99.1855,175.652 100.332,175.658 
 101.479,175.662 102.625,175.665 103.772,175.668 104.918,175.67 106.065,175.673 107.211,175.676 108.358,175.68 109.504,175.684 110.651,175.687 111.797,175.691 
 112.944,175.694 114.09,175.697 115.237,175.7 116.383,175.703 117.53,175.706 118.676,175.709 119.823,175.711 120.969,175.714 122.116,175.716 123.263,175.718 
 124.409,175.72 125.556,175.72 126.702,175.722 127.849,175.724 128.995,175.727 130.142,175.73 131.288,175.732 132.435,175.735 133.581,175.737 134.728,175.74 
 135.874,175.742 137.021,175.744 138.167,175.746 139.314,175.747 140.46,175.749 141.607,175.751 142.753,175.752 143.9,175.753 145.046,175.754 146.193,175.755 
 147.339,175.756 148.486,175.757 149.633,175.757 150.779,175.757 151.926,175.757 153.072,175.756 154.219,175.755 155.365,175.753 156.512,175.751 157.658,175.748 
 158.805,175.743 159.951,175.738 161.098,175.731 162.244,175.723 163.391,175.711 164.537,175.695 165.684,175.676 166.83,175.654 167.977,175.629 169.123,175.6 
 170.27,175.566 171.416,175.528 172.563,175.484 173.71,175.434 174.856,175.373 176.003,175.305 177.149,175.225 178.296,175.133 179.442,175.026 180.589,174.899 
 181.735,174.751 182.882,174.576 184.028,174.357 185.175,174.107 186.321,173.815 187.468,173.466 188.614,173.053 189.761,172.554 190.907,171.952 192.054,171.226 
 193.2,170.346 194.347,169.274 195.493,167.876 196.64,164.979 197.786,161.813 198.933,157.803 200.08,153.268 201.226,145.471 202.373,138.226 203.519,132.31 
 204.666,155.726 205.812,157.885 206.959,157.753 208.105,157.454 209.252,156.114 210.398,155.045 211.545,153.537 212.691,151.774 213.838,148.393 214.984,146.569 
 216.131,143.069 217.277,139.198 218.424,132.133 219.57,125.368 220.717,111.096 221.863,88.5295 223.01,66.2441 224.157,101.117 225.303,110.173 226.45,107.607 
 227.596,96.1449 228.743,82.6743 229.889,46.5787 231.036,24.1862 232.182,11.811 233.329,26.1123 234.475,101.71 235.622,109.956 236.768,113.875 237.915,112.467 
 239.061,109.177 240.208,106.218 241.354,102.989 242.501,94.9852 243.647,82.7129 244.794,69.6885 245.94,53.9825 247.087,60.0849 248.233,67.1435 249.38,68.5382 
 250.527,64.5632 251.673,58.7789 252.82,78.2634 253.966,100.874 255.113,108.244 256.259,112.048 257.406,112.335 258.552,112.364 259.699,111.476 260.845,108.598 
 261.992,105.062 263.138,100.349 264.285,96.3075 265.431,92.6805 266.578,82.0789 267.724,76.0275 268.871,80.5516 270.017,80.7792 271.164,78.4024 272.31,74.2797 
 273.457,70.4991 274.604,55.619 275.75,53.3662 276.897,67.4727 278.043,70.7432 279.19,70.9198 280.336,67.3519 281.483,64.4082 282.629,72.1481 283.776,81.7393 
 
 "/>
 
 
 
 
 ysmax 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 10000 
 
 
 15000 
 
 
 20000 
 
<polyline clip-path="url(#clip03)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 352.157,11.811 353.303,114.274 354.45,150.212 355.596,159.608 356.743,162.343 357.89,163.55 359.036,164.184 360.183,164.625 361.329,165.042 362.476,165.506 
 363.622,166.049 364.769,166.576 365.915,167.047 367.062,167.499 368.208,167.963 369.355,168.442 370.501,168.914 37

Fitting Done
True loglikelihood      = -Inf
Estimated loglikelihood = [-7168.56]


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,175.8 32.8746,175.801 33.5271,175.801 34.1796,175.802 34.8321,175.802 35.4846,175.802 36.1371,175.803 36.7895,175.803 37.442,175.804 38.0945,175.804 
 38.747,175.805 39.3995,175.805 40.052,175.805 40.7045,175.806 41.357,175.806 42.0095,175.806 42.662,175.807 43.3145,175.807 43.967,175.807 44.6194,175.807 
 45.2719,175.808 45.9244,175.808 46.5769,175.808 47.2294,175.808 47.8819,175.809 48.5344,175.809 49.1869,175.809 49.8394,175.809 50.4919,175.81 51.1444,175.81 
 51.7969,175.81 52.4493,175.81 53.1018,175.81 53.7543,175.81 54.4068,175.811 55.0593,175.811 55.7118,175.811 56.3643,175.811 57.0168,175.811 57.6693,175.811 
 58.3218,175.811 58.9743,175.812 59.6267,175.812 60.2792,175.812 60.9317,175.812 61.5842,175.812 62.2367,175.812 62.8892,175.812 63.5417,175.812 64.1942,175.812 
 64.8467,175.812 65.4992,175.813 66.1517,175.813 66.8042,175.813 67.4566,175.813 68.1091,175.813 68.7616,175.813 69.4141,175.813 70.0666,175.813 70.7191,175.813 
 71.3716,175.813 72.0241,175.813 72.6766,175.813 73.3291,175.813 73.9816,175.813 74.6341,175.813 75.2865,175.813 75.939,175.813 76.5915,175.813 77.244,175.813 
 77.8965,175.813 78.549,175.813 79.2015,175.813 79.854,175.813 80.5065,175.813 81.159,175.813 81.8115,175.813 82.464,175.813 83.1164,175.813 83.7689,175.813 
 84.4214,175.813 85.0739,175.813 85.7264,175.813 86.3789,175.813 87.0314,175.813 87.6839,175.813 88.3364,175.813 88.9889,175.813 89.6414,175.813 90.2938,175.813 
 90.9463,175.813 91.5988,175.813 92.2513,175.813 92.9038,175.813 93.5563,175.813 94.2088,175.813 94.8613,175.813 95.5138,175.813 96.1663,175.813 96.8188,175.813 
 97.4713,175.813 98.1237,175.813 98.7762,175.813 99.4287,175.813 100.081,175.813 100.734,175.813 101.386,175.812 102.039,175.812 102.691,175.812 103.344,175.812 
 103.996,175.812 104.649,175.812 105.301,175.812 105.954,175.812 106.606,175.812 107.259,175.811 107.911,175.811 108.564,175.811 109.216,175.811 109.869,175.811 
 110.521,175.811 111.174,175.81 111.826,175.81 112.479,175.81 113.131,175.81 113.784,175.81 114.436,175.809 115.089,175.809 115.741,175.809 116.394,175.808 
 117.046,175.808 117.698,175.808 118.351,175.807 119.003,175.807 119.656,175.807 120.308,175.806 120.961,175.806 121.613,175.805 122.266,175.805 122.918,175.804 
 123.571,175.803 124.223,175.803 124.876,175.802 125.528,175.801 126.181,175.8 126.833,175.799 127.486,175.798 128.138,175.797 128.791,175.796 129.443,175.794 
 130.096,175.792 130.748,175.79 131.401,175.787 132.053,175.784 132.706,175.78 133.358,175.775 134.011,175.768 134.663,175.759 135.316,175.747 135.968,175.729 
 136.621,175.703 137.273,175.664 137.926,175.604 138.578,175.509 139.231,175.356 139.883,175.101 140.536,174.662 141.188,173.861 141.841,172.248 142.493,168.286 
 143.146,154.448 143.798,91.3211 144.451,11.811 145.103,105.159 145.756,114.898 146.408,100.657 147.061,88.9107 147.713,81.7676 148.366,77.205 149.018,73.5326 
 149.671,70.4264 150.323,68.3923 150.976,67.9705 151.628,69.2703 152.281,71.9083 152.933,75.2071 153.586,78.4659 154.238,81.1668 154.891,83.0661 155.543,84.1876 
 156.195,84.749 156.848,85.0641 157.5,85.4439 158.153,86.119 158.805,87.1985 159.458,88.6659 160.11,90.4057 160.763,92.2488 161.415,94.0154 162.068,95.55 
 162.72,96.7398 163.373,97.5209 164.025,97.8743 164.678,97.8156 165.33,97.3861 165.983,96.6435 166.635,95.6521 167.288,94.4799 167.94,93.1931 168.593,91.8536 
 169.245,90.5175 169.898,89.2346 170.55,88.0473 171.203,86.9911 171.855,86.0937 172.508,85.3762 173.16,84.8521 173.813,84.529 174.465,84.408 175.118,84.4849 
 175.77,84.7505 176.423,85.1914 177.075,85.7909 177.728,86.5294 178.38,87.3854 179.033,88.3362 179.685,89.3585 180.338,90.4297 180.99,91.5267 181.643,92.6284 
 182.295,93.7151 18

In [51]:
demo(GenerateDatasets.MagdonIsmailAndAtiya(), [32], 20_000)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 1.5 
 
 
 2.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 42.4384,22.1496 43.6429,137.62 44.8473,148.515 46.0518,149.131 47.2563,147.339 48.4608,124.427 49.6652,79.1028 50.8697,40.2653 52.0742,19.289 53.2786,11.811 
 54.4831,12.1214 55.6876,17.2988 56.8921,25.9752 58.0965,37.342 59.301,50.752 60.5055,65.5807 61.71,81.1872 62.9144,96.921 64.1189,112.159 65.3234,126.344 
 66.5278,139.022 67.7323,149.86 68.9368,158.653 70.1413,165.343 71.3457,170.035 72.5502,172.315 73.7547,173.395 74.9591,174.066 76.1636,174.488 77.3681,174.762 
 78.5726,174.97 79.777,175.104 80.9815,175.217 82.186,175.299 83.3904,175.369 84.5949,175.422 85.7994,175.467 87.0039,175.511 88.2083,175.536 89.4128,175.568 
 90.6173,175.585 91.8217,175.609 93.0262,175.623 94.2307,175.641 95.4352,175.658 96.6396,175.668 97.8441,175.683 99.0486,175.689 100.253,175.701 101.458,175.705 
 102.662,175.715 103.866,175.726 105.071,175.727 106.275,175.736 107.48,175.741 108.684,175.745 109.889,175.752 111.093,175.753 112.298,175.759 113.502,175.766 
 114.707,175.765 115.911,175.77 117.116,175.775 118.32,175.776 119.525,175.781 120.729,175.78 121.934,175.784 123.138,175.789 124.342,175.787 125.547,175.791 
 126.751,175.794 127.956,175.793 129.16,175.797 130.365,175.796 131.569,175.799 132.774,175.799 133.978,175.801 135.183,175.804 136.387,175.804 137.592,175.804 
 138.796,175.807 140.001,175.807 141.205,175.807 142.41,175.808 143.614,175.808 144.819,175.807 146.023,175.81 147.227,175.812 148.432,175.809 149.636,175.812 
 150.841,175.808 152.045,175.81 153.25,175.812 154.454,175.81 155.659,175.812 156.863,175.813 158.068,175.812 159.272,175.812 160.477,175.813 161.681,175.812 
 162.886,175.813 164.09,175.81 165.295,175.811 166.499,175.813 167.703,175.811 168.908,175.812 170.112,175.81 171.317,175.812 172.521,175.811 173.726,175.809 
 174.93,175.811 176.135,175.808 177.339,175.809 178.544,175.808 179.748,175.809 180.953,175.806 182.157,175.807 183.362,175.804 184.566,175.805 185.771,175.804 
 186.975,175.805 188.179,175.802 189.384,175.802 190.588,175.803 191.793,175.801 192.997,175.801 194.202,175.797 195.406,175.798 196.611,175.799 197.815,175.796 
 199.02,175.797 200.224,175.794 201.429,175.794 202.633,175.794 203.838,175.791 205.042,175.792 206.247,175.789 207.451,175.789 208.656,175.787 209.86,175.787 
 211.064,175.784 212.269,175.784 213.473,175.781 214.678,175.781 215.882,175.781 217.087,175.78 218.291,175.778 219.496,175.775 220.7,175.775 221.905,175.771 
 223.109,175.771 224.314,175.771 225.518,175.77 226.723,175.768 227.927,175.765 229.132,175.765 230.336,175.761 231.54,175.76 232.745,175.759 233.949,175.757 
 235.154,175.754 236.358,175.753 237.563,175.75 238.767,175.749 239.972,175.748 241.176,175.745 242.381,175.744 243.585,175.741 244.79,175.74 245.994,175.736 
 247.199,175.736 248.403,175.733 249.608,175.729 250.812,175.728 252.016,175.723 253.221,175.722 254.425,175.721 255.63,175.718 256.834,175.714 258.039,175.713 
 259.243,175.709 260.448,175.707 261.652,175.703 262.857,175.702 264.061,175.696 265.266,175.695 266.47,175.69 267.675,175.688 268.879,175.685 270.084,175.683 
 271.288,175.678 272.493,175.674 273.697,175.671 274.901,175.667 276.106,175.663 277.31,175.658 278.515,175.655 279.719,175.651 280.924,175.647 282.128,175.644 
 
 "/>
 
 
 
 
 ysmax 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 800 
 
 
 850 
 
<polyline clip-path="url(#clip03)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 340.625,11.811 341.83,23.1222 343.034,23.2974 344.238,23.3992 345.443,26.5771 346.647,39.7495 347.852,48.1994 349.056,54.3851 350.261,59.2843 351.465,63.3697 
 352.67,66.8648 353.874,69.9005 355.079,72.5676 356.283,74.9364 357.488,77.0626 358.692,78.9935 359.897,80.7675 361.101,82.4178 362.306,83.9701 363.

Fitting Done
True loglikelihood      = -777.9679813457541
Estimated loglikelihood = [-780.578]


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -50 
 
 
 -25 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.8117,175.813 38.8659,175.813 38.92,175.813 38.9742,175.813 39.0283,175.813 39.0825,175.813 39.1366,175.813 39.1908,175.813 39.2449,175.813 39.2991,175.813 
 39.3532,175.813 39.4074,175.813 39.4615,175.813 39.5157,175.813 39.5698,175.813 39.624,175.813 39.6781,175.813 39.7323,175.813 39.7864,175.813 39.8406,175.813 
 39.8947,175.813 39.9489,175.813 40.003,175.813 40.0572,175.812 40.1113,175.812 40.1655,175.812 40.2196,175.812 40.2738,175.812 40.3279,175.812 40.3821,175.812 
 40.4362,175.812 40.4904,175.812 40.5445,175.812 40.5987,175.812 40.6528,175.812 40.707,175.812 40.7611,175.812 40.8153,175.812 40.8694,175.812 40.9236,175.812 
 40.9777,175.812 41.0319,175.812 41.086,175.812 41.1402,175.812 41.1943,175.811 41.2485,175.811 41.3027,175.811 41.3568,175.811 41.411,175.811 41.4651,175.811 
 41.5193,175.811 41.5734,175.811 41.6276,175.811 41.6817,175.811 41.7359,175.811 41.79,175.811 41.8442,175.811 41.8983,175.811 41.9525,175.811 42.0066,175.811 
 42.0608,175.811 42.1149,175.811 42.1691,175.811 42.2232,175.811 42.2774,175.81 42.3315,175.81 42.3857,175.81 42.4398,175.81 42.494,175.81 42.5481,175.81 
 42.6023,175.81 42.6564,175.81 42.7106,175.81 42.7647,175.81 42.8189,175.81 42.873,175.81 42.9272,175.81 42.9813,175.81 43.0355,175.81 43.0896,175.81 
 43.1438,175.81 43.1979,175.809 43.2521,175.809 43.3062,175.809 43.3604,175.809 43.4145,175.809 43.4687,175.809 43.5228,175.809 43.577,175.809 43.6311,175.809 
 43.6853,175.809 43.7394,175.809 43.7936,175.809 43.8477,175.809 43.9019,175.809 43.956,175.809 44.0102,175.809 44.0643,175.809 44.1185,175.808 44.1726,175.808 
 44.2268,175.808 44.2809,175.808 44.3351,175.808 44.3892,175.808 44.4434,175.808 44.4975,175.808 44.5517,175.808 44.6058,175.808 44.66,175.808 44.7141,175.808 
 44.7683,175.808 44.8224,175.808 44.8766,175.808 44.9307,175.807 44.9849,175.807 45.039,175.807 45.0932,175.807 45.1473,175.807 45.2015,175.807 45.2556,175.807 
 45.3098,175.807 45.3639,175.807 45.4181,175.807 45.4722,175.807 45.5264,175.807 45.5805,175.807 45.6347,175.807 45.6888,175.806 45.743,175.806 45.7971,175.806 
 45.8513,175.806 45.9054,175.806 45.9596,175.806 46.0137,175.806 46.0679,175.806 46.122,175.806 46.1762,175.806 46.2303,175.806 46.2845,175.806 46.3386,175.806 
 46.3928,175.805 46.4469,175.805 46.5011,175.805 46.5552,175.805 46.6094,175.805 46.6635,175.805 46.7177,175.805 46.7718,175.805 46.826,175.805 46.8801,175.805 
 46.9343,175.805 46.9884,175.805 47.0426,175.805 47.0967,175.804 47.1509,175.804 47.205,175.804 47.2592,175.804 47.3133,175.804 47.3675,175.804 47.4216,175.804 
 47.4758,175.804 47.5299,175.804 47.5841,175.804 47.6382,175.804 47.6924,175.804 47.7465,175.803 47.8007,175.803 47.8548,175.803 47.909,175.803 47.9631,175.803 
 48.0173,175.803 48.0714,175.803 48.1256,175.803 48.1797,175.803 48.2339,175.803 48.288,175.803 48.3422,175.802 48.3963,175.802 48.4505,175.802 48.5046,175.802 
 48.5588,175.802 48.6129,175.802 48.6671,175.802 48.7212,175.802 48.7754,175.802 48.8295,175.802 48.8837,175.801 48.9378,175.801 48.992,175.801 49.0461,175.801 
 49.1003,175.801 49.1544,175.801 49.2086,175.801 49.2627,175.801 49.3169,175.801 49.371,175.801 49.4252,175.8 49.4793,175.8 49.5335,175.8 49.5876,175.8 
 49.6418,175.8 49.696,175.8 49.7501,175.8 49.8043,175.8 49.8584,175.8 49.9126,175.8 49.9667,175.799 50.0209,175.799 50.075,175.799 50.1292,175.799 
 50.1833,175.799 50.2375,175.799 50.2916,175.799 50.3458,175.799 50.3999,175.799 50.4541,175.798 50.5082,175.798 50.5624,175.798 50.6165,175.798 50.6707,175.798 
 50.7248,175.798 50.779,175.798 50.8331,175.798 50.8873,175.797 50.9414,175.797 50.9956,175.797 51.0497,175.797 51.1039,175.797 51.158,175.797 51.2122,175.797 
 51.2663,175.797 51.32

In [32]:
demo(Arcsine(1,4), [64,64], 20_000)

loglikelihood(dataset, data) = -4297.03876965193


2017-09-18 18:18:16.054638: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0)


(ii, ysmin, ysmax, condition_loss) = (1, [1.0028], [1.0028], [0.99442])
(ii, ysmin, ysmax, condition_loss) = (51, [1.1674], [1.16745], [0.721155])
(ii, ysmin, ysmax, condition_loss) = (101, [1.5134], [1.51683], [0.49703])
(ii, ysmin, ysmax, condition_loss) = (151, [1.50383], [1.51329], [0.490727])
(ii, ysmin, ysmax, condition_loss) = (201, [1.50517], [1.52705], [0.478882])
(ii, ysmin, ysmax, condition_loss) = (251, [1.50383], [1.54495], [0.460916])
(ii, ysmin, ysmax, condition_loss) = (301, [1.50064], [1.56139], [0.443022])
(ii, ysmin, ysmax, condition_loss) = (351, [1.49871], [1.57901], [0.425944])
(ii, ysmin, ysmax, condition_loss) = (401, [1.49686], [1.59772], [0.408694])
(ii, ysmin, ysmax, condition_loss) = (451, [1.49421], [1.61673], [0.391135])
(ii, ysmin, ysmax, condition_loss) = (501, [1.49064], [1.63635], [0.372975])
(ii, ysmin, ysmax, condition_loss) = (551, [1.48595], [1.65701], [0.353791])
(ii, ysmin, ysmax, condition_loss) = (601, [1.47983], [1.67906], [0.333237])
(ii, ysm

LoadError: [91mInterruptException:[39m

In [17]:
dataset = GenerateDatasets.Likas1()
data = original_sample(dataset)
@show loglikelihood(dataset, data)
est = NeuralDensityEstimator([64], approximate_support(dataset))
condition!(est)
println("Conditioning Done")
fit!(est, data; epochs=10_000)
println("Fitting Done")

loglikelihood(dataset, data) = -10459.84346226728


2017-09-18 18:11:22.737611: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0)


LoadError: [91mInterruptException:[39m

In [18]:
gr =est.sess.graph
run(est.sess, exp(6*gr["W_2"].^2 + gr["b_2"]))

LoadError: [91mUndefVarError: est not defined[39m

In [19]:
collect(keys(est.sess.graph))

LoadError: [91mUndefVarError: est not defined[39m